# 1. 고객님의 음성을 녹음합니다!

조금 어려운 부분입니다.. 음성 녹음을 콜랩에서 돌릴 수 있도록 이것 저것 설치합니다! 좀 오래 걸릴 수 있습니다!

In [1]:
# 필요한 패키지
!pip install --quiet git+https://github.com/openai/whisper.git
!pip install --quiet sounddevice wavio
!pip install --quiet ipywebrtc notebook
!apt install --quiet ffmpeg
!apt-get install --quiet libportaudio2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.0 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubu

In [2]:
import os
import numpy as np

try:
    import tensorflow
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

from google.colab import output
output.enable_custom_widget_manager()

In [3]:
!pip install --quiet openai
import openai
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('op')
openai.api_key = os.getenv("OPENAI_API_KEY")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [5]:
client = openai.OpenAI()
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="사는게 참 힘든데 내가 참 고등학교로 돌아간다면은 공부를 좀 더 열심히 할 것 같아.",
)


response.stream_to_file("output.mp3")

<ipython-input-5-57147b0e28c7>:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.mp3")


# 2. 고객님의 음성을 텍스트로 변환합니다!

In [6]:
audio_file= open("output.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)
# 내용 봅시다!
print(transcription.text)

사는 게 참 힘든데 내가 참 고등학교로 돌아간다면 공부를 좀 더 열심히 할 것 같아.


# 3. 혹시 진상 고객님은 아닌지 확인해 봅니다...

In [ ]:
# Moderation code

In [7]:
response = client.moderations.create(input=transcription.text)
output = response.results[0]
output.to_dict()

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'hate': False,
  'hate/threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'sexual': False,
  'sexual/minors': False,
  'violence': False,
  'violence/graphic': False},
 'category_scores': {'harassment': 0.00010971521987812594,
  'harassment/threatening': 4.7909393288136926e-06,
  'hate': 1.8877746015277808e-06,
  'hate/threatening': 2.051384839774073e-08,
  'self-harm': 1.214439998875605e-05,
  'self-harm/instructions': 1.4625766198150814e-06,
  'self-harm/intent': 4.989011358702555e-05,
  'sexual': 0.00010072552686324343,
  'sexual/minors': 4.0011345845414326e-05,
  'violence': 0.00015975510177668184,
  'violence/graphic': 5.616051566903479e-06},
 'flagged': False}

# 4. 진상 고객이 아니시네요! 고객님의 소원을 해결해드립시다!

In [ ]:
# @title
from openai import OpenAI
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

client = OpenAI()

# text-embedding-ada 불러서 임베딩 가져오는 펑션
def get_embeddings(text):
  return client.embeddings.create(
    model="text-embedding-ada-002", #제일 무난하게 사용하는 임베딩 모델 추천
    input=text,
    encoding_format="float"
  ).data[0].embedding


# 임베딩을 만들어봅시다!
no_mon_embed      = get_embeddings("돈이 없다")
timetravel_embed  = get_embeddings("회귀")
bodysnat_embed    = get_embeddings("빙의")
reincarn_embed    = get_embeddings("환생")

# 미니 VectorDB 를 만들어봅시다..
solutions ={
    "돈이 없다": {
        "embeddings": no_mon_embed,
        "solutions": ["돈 있는 캐릭터가 된다!"]
    },
    "회귀": {
        "embeddings": timetravel_embed,
        "solutions": ["시간을 거슬러 되돌아 감!"]
    },
    "빙의": {
        "embeddings": bodysnat_embed,
        "solutions": ["소설속 캐릭터로 빙의하자!"]
    },
    "환생": {
        "embeddings": reincarn_embed,
        "solutions": ["판타지 소설의 캐릭터로 다시 태어나자!"]
    }
}

def get_highest_match(problem_embed):
    # 사용자의 문제를 받고, 우리의 미니 DB 에서 제일 가까운 솔루션을 찾아낼 것입니다!
    results = []
    for key, value in solutions.items():
        sim = cosine_similarity([problem_embed], [value['embeddings']])[0][0]
        results.append((key, sim, value['solutions']))

    # 유사도로 정리
    results.sort(key=lambda x: x[1], reverse=True)

    # 제일 유사한 하나만 가져오기
    return results[0]

def get_solution(problem) -> str:
    # 사용자의 문제를 임베딩으로 바꿈
    problem_embed = get_embeddings(problem)

    # 제일 가까운 카테고리의 솔루션을 가지고 옴
    solution_found = get_highest_match(problem_embed)[2][0] #(key, sim, value['solutions']) 이런 형태니까 솔루션만 가져온듯
    return solution_found

def get_problem(user_query):
  # 사용자의 질문을 듣고 카테고리를 선별함
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """사용자의 질문을 듣고 그 문제를 해결할 수 있는 방법을 다음 다섯가지 중에서 선택해줘:
            '돈이 없다', '회귀', '빙의', '환생'
            이 중에 선택할 옵션이 없으면 그냥 아무 대답도 하지 마.
            """
            },
            {"role": "user", "content": user_query},
        ]
    ).choices[0].message.content

def get_detailed_solution(user_query):
  # 모든 자료가 모였으니 이제 마지막 답변을 생성함
  solution = get_solution(get_problem(user_query))
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """너는 퐌타스틱 스토리 작가 AI이다. 너의 역할은 주변에서 흔히 일어날 수 있는 문제를 가진 사람의 불평을 기본으로,
            판타지스러운 설정을 만들어내는 것이다. 이 스토리 안에서 너는 현대인이 길에서 발견하는 작은 디바이스이며, 그 사람이 무언가에 대해 불평하면
            그 판타지스러운 설정을 제안하면 된다.

            예:
            사람: "아, 난 이번 생은 망한 것 같아."
            AI: "그렇다면 판타지 소설속의 인물로 환생하는 것은 어떨까요? 부유한 백작가의 막내 아들로 말이죠!

            """
            },
            {"role": "user", "content": user_query},
            {"role": "system", "content": f"""
            이번 사용자의 불평은 {user_query} 이며, 이것에 대한 해결법은 {solution}이다.
            이전의 예시를 참고하여 사용자에게 그럴듯한 판타지 설정을 제안해라.
            판타지 설정에는 가상의 나라, 그리고 황족, 왕족, 귀족 등의 신분, 기사나 사제, 상단주 혹은 마법사 같은 직업이 필요하다.
            머리색깔과 눈색깔도 여러가지로 설정할 수 있다. 예: 빨강, 검정, 금색, 은색, 파랑, 녹색, 보라색, 자주색, 갈색.
            주인공이 사는 가상의 나라와, 가상의 신분과, 가상의 직업과, 머리색깔과 눈색깔을 정하고, 신분과 직업에 기반한 스토리 라인을 정한다.
            기본적으로 시작할 때에는 불행하나 훌륭한 능력과 운으로 크게 성공한다는 줄거리로 200자 이상의 내용을 만들어 보낸다.

            판타지 설정:

            """
            },
        ]
    ).choices[0].message.content

In [ ]:
answer = get_detailed_solution(transcription.text)
#이건 실행 안함 위에 코드에서 보이듯이 토큰이 너무나도 길어짐... 절약하면서 공부해야지
answer

'"그렇다면, 어떤가요? \'시간을 거슬러가는 마법사\'로서의 새로운 삶을 시작하는 것은. 당신은 \'테라노스\'라는 이름의 아름다운 판타지 세계에서 은빛 머리와 청록색 눈을 가진 젊은 마법사로 살게 됩니다. \n\n당신은 고등학교 시절로 돌아가, 그때의 모든 것을 다시 경험하게 됩니다. 하지만 이번에는 달라요. 당신은 마법사로서의 지식과 능력을 가지고 있습니다. 당신은 공부에 힘쓰는 것이 아니라, 마법을 이용해 문제를 해결하게 됩니다. \n\n당신은 고등학교 시절의 친구들에게 마법을 가르치며, 그들과 함께 모험을 떠나게 됩니다. 당신은 마법사로서의 신분을 이용해, 고등학교 시절의 문제들을 해결하고, 그 과정에서 자신만의 능력을 발견하게 됩니다. \n\n당신은 마법사로서의 삶을 즐기며, 고등학교 시절의 자신을 돌아보게 됩니다. 그리고 당신은 결국, 마법사로서의 삶이 자신에게 주어진 두 번째 기회라는 것을 깨닫게 됩니다. 이번에는 공부에만 집중하는 것이 아니라, 자신의 능력을 최대한 발휘하며 삶을 즐기는 것이 중요하다는 것을 깨닫게 됩니다."'

# 5. 답변 퀄리티 한 번 볼게요..

In [ ]:
# Groundedness test
#이 부분은 코드가 없다?? RAGAS를 이용해서 평가를 해볼까 했는데 의문이 들음
#Groun truth를 뭐로 해야하지?? 이건 정답이 없는 답변이라 애매하다.


# 6. 끝났으니 이제는 음성으로 바꿔서 보냅시다!

In [ ]:
speech_file_path = "speech_output.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="nova",
  input=answer
)
response.stream_to_file(speech_file_path)

<ipython-input-20-52d0a520c935>:7: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [ ]:
from IPython.display import Audio, display
Audio("speech_output.mp3")  # From URL


In [8]:
a=client.images.generate(
    model="dall-e-3",
    prompt="그렇다면, 어떤가요? \'시간을 거슬러가는 마법사\'로서의 새로운 삶을 시작하는 것은. 당신은 \'테라노스\'라는 이름의 아름다운 판타지 세계에서 은빛 머리와 청록색 눈을 가진 젊은 마법사로 살게 됩니다. \n\n당신은 고등학교 시절로 돌아가, 그때의 모든 것을 다시 경험하게 됩니다. 하지만 이번에는 달라요. 당신은 마법사로서의 지식과 능력을 가지고 있습니다. 당신은 공부에 힘쓰는 것이 아니라, 마법을 이용해 문제를 해결하게 됩니다. \n\n당신은 고등학교 시절의 친구들에게 마법을 가르치며, 그들과 함께 모험을 떠나게 됩니다. 당신은 마법사로서의 신분을 이용해, 고등학교 시절의 문제들을 해결하고, 그 과정에서 자신만의 능력을 발견하게 됩니다. \n\n당신은 마법사로서의 삶을 즐기며, 고등학교 시절의 자신을 돌아보게 됩니다. 그리고 당신은 결국, 마법사로서의 삶이 자신에게 주어진 두 번째 기회라는 것을 깨닫게 됩니다. 이번에는 공부에만 집중하는 것이 아니라, 자신의 능력을 최대한 발휘하며 삶을 즐기는 것이 중요하다는 것을 깨닫게 됩니다.",
    size="1024x1024",
    quality="standard",
    n=1
)

image_url=a.data[0].url

In [9]:
from IPython.display import display, HTML
html=f'<img src="{image_url}" width="600">'
display(HTML(html))